In [1]:
#import libraries
import pandas as pd
import json
import re
import os
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import pickle
import matplotlib.pyplot as plt
import requests
import warnings
import pprint
warnings.simplefilter('ignore')

In [2]:
#read all files
for file in os.listdir("./all_recipes"):
    print(file)

recipe1.csv
recipe2.csv
recipe3.csv
recipe4.csv
recipe5.csv
recipe6.csv
recipe7.csv
recipe8.csv
recipe9.csv


In [3]:
#store all csv into a dataframe
df_list = []
for file in os.listdir("./all_recipes"):
    if file.endswith(".csv"):
        recipe_df = pd.read_csv(f"./all_recipes/{file}")#, sep=";")#, parse_dates=True)
        df_list.append(recipe_df)
    recipe_df = pd.concat(df_list, sort=True)

In [4]:
recipe_df.head()

,cuisines,dish,grocery,id,image,ingredients,instructions,readyInMinutes,servings,title
0,Italian,1,"['broccolini', 'garlic clove', 'olive oil', 'o...",715769,https://spoonacular.com/recipeImages/715769-31...,"{'broccolini': '0.5 bunch', 'garlic clove': '0...",In a large pan with lid heat olive oil over me...,30,2,Broccolini Quinoa Pilaf
1,Italian,1,"['artichoke', 'bay leaf', 'coriander seeds', '...",648257,https://spoonacular.com/recipeImages/648257-31...,"{'artichoke': '1.0 pound', 'bay leaf': '1.0 ',...",Snip the thorns off the artichoke leaves.Place...,35,1,Italian Steamed Artichokes
2,Italian,1,"['crusty bread', 'mushrooms', 'spinach', 'chic...",1095745,https://spoonacular.com/recipeImages/1095745-3...,"{'crusty bread': '0.25 loaf', 'mushrooms': '0....","To make the hummus, place all ingredients in a...",30,4,Mushroom Hummus Crostini
3,Italian,1,"['balsamic vinegar', 'extra virgin olive oil',...",658753,https://spoonacular.com/recipeImages/658753-31...,"{'balsamic vinegar': '0.5 tablespoons', 'extra...",Slice the bread on a bias about 1/2 inch thick...,45,4,Roma Tomato Bruschetta
4,Italian,1,"['balsamic vinegar', 'brussels sprouts', 'extr...",658515,https://spoonacular.com/recipeImages/658515-31...,"{'balsamic vinegar': '0.25 tablespoon', 'bruss...",Heat oven to 450 degrees. Trim bottom of Bruss...,45,4,Roasted Brussels Sprouts With Garlic


In [5]:
recipe_df.columns

Index(['cuisines', 'dish', 'grocery', 'id', 'image', 'ingredients',
       'instructions', 'readyInMinutes', 'servings', 'title'],
      dtype='object')

In [6]:
recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 696 entries, 0 to 98
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cuisines        696 non-null    object
 1   dish            696 non-null    int64 
 2   grocery         696 non-null    object
 3   id              696 non-null    int64 
 4   image           696 non-null    object
 5   ingredients     696 non-null    object
 6   instructions    695 non-null    object
 7   readyInMinutes  696 non-null    int64 
 8   servings        696 non-null    int64 
 9   title           696 non-null    object
dtypes: int64(4), object(6)
memory usage: 59.8+ KB


In [7]:
recipe_df.fillna(value=0,inplace=True)
recipe_df.head()

,cuisines,dish,grocery,id,image,ingredients,instructions,readyInMinutes,servings,title
0,Italian,1,"['broccolini', 'garlic clove', 'olive oil', 'o...",715769,https://spoonacular.com/recipeImages/715769-31...,"{'broccolini': '0.5 bunch', 'garlic clove': '0...",In a large pan with lid heat olive oil over me...,30,2,Broccolini Quinoa Pilaf
1,Italian,1,"['artichoke', 'bay leaf', 'coriander seeds', '...",648257,https://spoonacular.com/recipeImages/648257-31...,"{'artichoke': '1.0 pound', 'bay leaf': '1.0 ',...",Snip the thorns off the artichoke leaves.Place...,35,1,Italian Steamed Artichokes
2,Italian,1,"['crusty bread', 'mushrooms', 'spinach', 'chic...",1095745,https://spoonacular.com/recipeImages/1095745-3...,"{'crusty bread': '0.25 loaf', 'mushrooms': '0....","To make the hummus, place all ingredients in a...",30,4,Mushroom Hummus Crostini
3,Italian,1,"['balsamic vinegar', 'extra virgin olive oil',...",658753,https://spoonacular.com/recipeImages/658753-31...,"{'balsamic vinegar': '0.5 tablespoons', 'extra...",Slice the bread on a bias about 1/2 inch thick...,45,4,Roma Tomato Bruschetta
4,Italian,1,"['balsamic vinegar', 'brussels sprouts', 'extr...",658515,https://spoonacular.com/recipeImages/658515-31...,"{'balsamic vinegar': '0.25 tablespoon', 'bruss...",Heat oven to 450 degrees. Trim bottom of Bruss...,45,4,Roasted Brussels Sprouts With Garlic


In [8]:
recipe_df.rename(columns={'grocery':'ingredients_lists'},inplace=True)
#recipe_df

# Read the Nutrition Data

In [9]:
new_nutrition_df=pd.read_csv('./final/new_nutrition.csv')
new_nutrition_df.head()

,recipe_id,Carbohydrates g,Calories kcal,Fat g,Protein g
0,715769,71.66,625.07,30.69,19.86
1,648257,49.50,221.45,0.82,15.21
2,1095745,76.92,468.07,9.76,21.49
3,658753,67.36,466.44,16.31,13.70
4,658515,16.15,232.48,18.01,5.82


In [10]:
new_nutrition_df.rename(columns={'recipe_id':'id'},inplace=True)
new_nutrition_df.head(3)

,id,Carbohydrates g,Calories kcal,Fat g,Protein g
0,715769,71.66,625.07,30.69,19.86
1,648257,49.50,221.45,0.82,15.21
2,1095745,76.92,468.07,9.76,21.49


In [11]:
##Read the Ingredients Data

In [12]:
#new_ingredients_df=pd.read_csv('./final/all_ingredients.csv')
#new_ingredients_df

In [13]:
#new_ingredients_df.rename(columns={'recipe_id':'id'},inplace=True)
#new_ingredients_df.set_index('id',inplace=True)
#new_ingredients_df

# Merge Two DataFrames

In [14]:
all_recipes_df=recipe_df.merge(new_nutrition_df,on='id')
all_recipes_df.head()

,cuisines,dish,ingredients_lists,id,image,ingredients,instructions,readyInMinutes,servings,title,Carbohydrates g,Calories kcal,Fat g,Protein g
0,Italian,1,"['broccolini', 'garlic clove', 'olive oil', 'o...",715769,https://spoonacular.com/recipeImages/715769-31...,"{'broccolini': '0.5 bunch', 'garlic clove': '0...",In a large pan with lid heat olive oil over me...,30,2,Broccolini Quinoa Pilaf,71.66,625.07,30.69,19.86
1,Italian,1,"['broccolini', 'garlic clove', 'olive oil', 'o...",715769,https://spoonacular.com/recipeImages/715769-31...,"{'broccolini': '0.5 bunch', 'garlic clove': '0...",In a large pan with lid heat olive oil over me...,30,2,Broccolini Quinoa Pilaf,71.66,625.07,30.69,19.86
2,Italian,1,"['broccolini', 'garlic clove', 'olive oil', 'o...",715769,https://spoonacular.com/recipeImages/715769-31...,"{'broccolini': '0.5 bunch', 'garlic clove': '0...",In a large pan with lid heat olive oil over me...,30,2,Broccolini Quinoa Pilaf,71.66,625.07,30.69,19.86
3,Italian,1,"['broccolini', 'garlic clove', 'olive oil', 'o...",715769,https://spoonacular.com/recipeImages/715769-31...,"{'broccolini': '0.5 bunch', 'garlic clove': '0...",In a large pan with lid heat olive oil over me...,30,2,Broccolini Quinoa Pilaf,71.66,625.07,30.69,19.86
4,Italian,1,"['broccolini', 'garlic clove', 'olive oil', 'o...",715769,https://spoonacular.com/recipeImages/715769-31...,"{'broccolini': '0.5 bunch', 'garlic clove': '0...",In a large pan with lid heat olive oil over me...,30,2,Broccolini Quinoa Pilaf,71.66,625.07,30.69,19.86


In [15]:
#all_recipes_df=recipe_df.merge(new_ingredients_df,on='id')
#all_recipes_df

In [16]:
##Drop Duplicates

In [17]:
all_recipes_df.drop_duplicates(subset ="title",keep ='last', inplace = True)
all_recipes_df.head()

,cuisines,dish,ingredients_lists,id,image,ingredients,instructions,readyInMinutes,servings,title,Carbohydrates g,Calories kcal,Fat g,Protein g
8,"['Mediterranean', 'Italian', 'European']",1,"['broccolini', 'garlic clove', 'olive oil', 'o...",715769,https://spoonacular.com/recipeImages/715769-31...,"{'broccolini': '0.5 bunch', 'garlic clove': '0...",In a large pan with lid heat olive oil over me...,30,2,Broccolini Quinoa Pilaf,71.66,625.07,30.69,19.86
12,Italian,1,"['artichoke', 'bay leaf', 'coriander seeds', '...",648257,https://spoonacular.com/recipeImages/648257-31...,"{'artichoke': '1.0 pound', 'bay leaf': '1.0 ',...",Snip the thorns off the artichoke leaves.Place...,35,1,Italian Steamed Artichokes,49.50,221.45,0.82,15.21
16,Italian,1,"['crusty bread', 'mushrooms', 'spinach', 'chic...",1095745,https://spoonacular.com/recipeImages/1095745-3...,"{'crusty bread': '0.25 loaf', 'mushrooms': '0....","To make the hummus, place all ingredients in a...",30,4,Mushroom Hummus Crostini,76.92,468.07,9.76,21.49
20,Italian,1,"['balsamic vinegar', 'extra virgin olive oil',...",658753,https://spoonacular.com/recipeImages/658753-31...,"{'balsamic vinegar': '0.5 tablespoons', 'extra...",Slice the bread on a bias about 1/2 inch thick...,45,4,Roma Tomato Bruschetta,67.36,466.44,16.31,13.70
24,Italian,1,"['balsamic vinegar', 'brussels sprouts', 'extr...",658515,https://spoonacular.com/recipeImages/658515-31...,"{'balsamic vinegar': '0.25 tablespoon', 'bruss...",Heat oven to 450 degrees. Trim bottom of Bruss...,45,4,Roasted Brussels Sprouts With Garlic,16.15,232.48,18.01,5.82


In [18]:
#final_df=all_recipes_df.drop(columns=['cuisines','ingredients_lists','ingredients','title','image'])
#final_df.set_index('id',inplace=True)
#final_df

In [19]:
#final_df.to_csv("./final/final_df.csv",index=True)

In [20]:
all_recipes_df.to_csv("./final/all_recipes_new_df.csv",index=True)